In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Daily Capital Flow Calculation with RDD").getOrCreate()
rdd = spark.read.csv("user_balance_table.csv", header=True, inferSchema=True).rdd
mapped_rdd = rdd.map(lambda row: (row["report_date"], (row["total_purchase_amt"], row["total_redeem_amt"])))
aggregated_rdd = mapped_rdd.reduceByKey(lambda a, b: (a[0] + b[0], a[1] + b[1]))
sorted_rdd = aggregated_rdd.sortByKey()
result_rdd = sorted_rdd.map(lambda x: (x[0], x[1][0], x[1][1]))
result_rdd.saveAsTextFile("task1_result")
spark.stop()


In [2]:
from pyspark.sql import SparkSession
from datetime import datetime

spark = SparkSession.builder.appName("Active Users Calculation").getOrCreate()
file_path = "user_balance_table.csv"  
rdd = spark.read.csv(file_path, header=True, inferSchema=True).rdd

filtered_rdd = rdd.filter(lambda row: datetime.strptime(str(row["report_date"]), "%Y%m%d").strftime("%Y%m") == "201408").map(lambda row: (row["user_id"], row["report_date"]))

user_active_days = filtered_rdd.distinct()
user_active_days_count = user_active_days.mapValues(lambda x: 1).reduceByKey(lambda a, b: a + b)

active_users = user_active_days_count.filter(lambda x: x[1] >= 5)
active_user_count = active_users.count()
print(active_user_count)
spark.stop()

12767
